In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re



In [9]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding="ISO-8859-1",names=["target", "ids", "date", "flag", "user", "text"])

pos100 = df[:1_000]
neg100 = df[df["target"] == 4][:1_000]
smaller_data = pd.concat([pos100,neg100])
smaller_data.reset_index(inplace=True)
smaller_data = smaller_data.drop(["ids","date", "flag", "user", "index"], axis=1)
del(df)
smaller_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [6]:
import re
def remove_handles (text):
  return re.sub("@[^\s]+", "", text)

def remove_link(text):
  return re.sub(r'http\S+', '', text)
  
def lower_case(text):
  return text.lower()

def remove_chars(text):
  text  = "".join([char for char in text if char not in string.punctuation]) #do we want to replace with space?
  text = re.sub('[0-9]+', '', text)
  return text

def tokenize(text): 
    text = re.split('\W+', text)
    return text

def remove_stop_words(words): #works on tokenized data
  stopword = nltk.corpus.stopwords.words('english')
  stopword.append("")
  return [word for word in words if word not in stopword]

def stemming(text):
  ps = nltk.SnowballStemmer('english')
  text = [ps.stem(word) for word in text]
  return text

def clean_and_tokenize(text):
  text = remove_handles(text)
  text = remove_link(text)
  text = lower_case(text)
  text = remove_chars(text)
  text = tokenize(text)
  text = remove_stop_words(text)
  text = stemming(text)
  return text


In [10]:
smaller_data["cleaned"] = smaller_data["text"].apply(clean_and_tokenize)

In [11]:
smaller_data["cleaned"].head()

0    [awww, that, bummer, shoulda, got, david, carr...
1    [upset, cant, updat, facebook, text, might, cr...
2    [dive, mani, time, ball, manag, save, rest, go...
3               [whole, bodi, feel, itchi, like, fire]
4                          [behav, im, mad, cant, see]
Name: cleaned, dtype: object

In [14]:

def generate_dictionary (df):
  dictionary = {}
  i = 0
  for row in df["cleaned"]:
    for word in row:
      if word not in dictionary.keys():
        dictionary[word] = i
        i += 1
  return dictionary

In [15]:
dictionary = generate_dictionary(smaller_data)
len(dictionary)

4052

In [16]:
def create_feature_matrix(df, dictionary):
  X=np.zeros((len(df), len(dictionary)))
  for index, row in enumerate(df["cleaned"]):
    for word in row:
      if word in dictionary.keys():
        col = dictionary[word]
        X[index, col] = 1
  return X    

In [17]:
X = create_feature_matrix(smaller_data, dictionary)
X.shape

(2000, 4052)

In [18]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


cv = KFold(n_splits=5, random_state=2, shuffle=True)
# create model
model = LogisticRegression(solver='lbfgs', max_iter=1000 )
# evaluate model
scores = cross_val_score(model, X, smaller_data["target"], scoring='accuracy', cv=cv)
sum(scores) / len(scores)

0.6729999999999999